In [10]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [2]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from IPython.display import display, clear_output
from fastai.data.all import *
from fastai.distributed import *
from fastscript import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *

ModuleNotFoundError: No module named 'video_block'

In [3]:
#exporti
def read_data(eugenio):
    path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv' if eugenio else '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()#.sample(n=5000)
    return df 

def get_vid_path(df:pd.Series): return Video(df['paths'].split('\n'))
def get_lbls(df:pd.Series): return df['lbl']

@call_parse
def main(gpu    :Param("GPU to run on", int)=None,
         eugenio:Param("Variable to control whether is Eugenio or me that's calling the trianing", bool)=False,
         n_lbl  :Param("# of different labels per batch", int)=8,
         n_el   :Param("# of elements per label", int)=4,
         l      :Param("Target number of frames of the ResizeTime transform", int)=60
        ):
    
    if gpu is not None: 
        gpu = setup_distrib(gpu)    
        df = rank0_first(lambda: read_data(eugenio)) 
    else:
        df = read_data(eugenio)

    tfms = [[get_vid_path, ResizeTime(l=l), Video.create],
            [get_lbls, Categorize()]]
    splits = RandomSplitter(seed=42)(df)
    splits = [uniformize_dataset(idxs, df.iloc[idxs]['lbl'] , n_lbl=n_lbl, n_el=n_el) for idxs in splits]
    
    dsets = Datasets(df, tfms, splits=splits)

    dls = dsets.dataloaders(bs=n_el*n_lbl,
                            shuffle_train=False,
                            after_item=[Resize(112), ToTensor()],
                            after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

    head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
    learn = Learner(dls, 
                    TLModel(body,head),
                    loss_func=MixedLoss(alpha=1.0, margin=.2),
                    splitter=my_splitter,
                    metrics=tl_accuracy)
    
    torch.cuda.empty_cache()
#     if gpu is not None:
#         with learn.distrib_ctx(gpu):
#             learn.fine_tune(4)
#     else:
#         with learn.parallel_ctx(device_ids=[0,1]):
    learn.fine_tune(4)
            
    learn.save('trained_models/Inflated1')

In [8]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_distributed_inflated_NN.ipynb.
Converted 05_first_inflated_NN.ipynb.
Converted index.ipynb.


In [6]:
# default_exp first_inflated_NN

In [ ]:
nn.Trans